In [1]:
import pandas as pd


In [2]:
# contains embeddings -> average across the window ? 
df_train = pd.read_csv("/home/d/repos/protein_properties/data/LMNglyPred/df_train_data_without_independent_test_and_protein.csv")
df_train["PID"]

,label,PID,Position,Sequence,Middle_Amino_Acid_ASN(N),1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,1,Q13425,167,RALRLGDAILSVNGTDLRQATHDQA,N,0.214431,0.070759,0.057231,-0.380399,0.193821,...,0.253808,-0.338158,-0.130401,0.153775,0.035714,-0.232914,-0.020075,0.076612,0.224596,-0.082093
1,1,Q9NZR2,1921,TSFAVGIDFHAENDTIYWTDMGFNK,N,0.123560,-0.119222,-0.159496,-0.024361,-0.033037,...,-0.279450,-0.297205,-0.013744,-0.109153,0.357152,-0.272427,0.124468,0.174194,0.521426,0.257090
2,1,O43663,489,LAPNTPGKARKLNTTTMSNATANSS,N,0.275733,0.100278,0.315226,0.036384,-0.026312,...,0.058070,-0.220898,0.285270,-0.010973,0.134187,-0.184231,0.042180,-0.179634,0.274765,0.186400
3,1,Q495M3,181,LADNLKQVVEAVNSTTNNCYSNETV,N,-0.108271,0.025474,-0.013394,-0.074594,0.315404,...,-0.082117,-0.331716,0.313081,-0.018301,0.105079,-0.464843,-0.085850,0.283040,0.404614,0.359031
4,1,Q9BXP8,1408,CPSLLLDHADVVNCTSIGPGLMKCA,N,-0.171608,-0.055078,-0.003691,0.201869,0.053884,...,-0.160630,-0.131732,0.083282,-0.149677,0.240220,-0.294894,-0.061772,0.204904,0.678788,0.174710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204,1,Q16849,524,PALTFRIRHNEQNLSLADVTQQAGL,N,-0.081075,-0.047008,-0.042428,-0.260029,0.294536,...,-0.292105,-0.187284,0.054134,-0.181451,0.251356,-0.290204,-0.127871,0.261313,0.198928,0.191233
1205,1,Q02539,79,KALAAAGYDVEKNNSRIKLGIKSLV,N,-0.359557,0.124104,0.058495,0.181729,0.360909,...,-0.378161,-0.463138,-0.169644,-0.113119,0.224437,0.181762,-0.264302,-0.109938,0.217059,0.282482
1206,1,Q8IZT6,3325,MEVIRYAVQVLLNVSKYEKTTSAVY,N,-0.197014,0.106527,-0.059967,-0.256705,0.178624,...,-0.490049,-0.246876,-0.086499,0.171711,0.191005,0.384794,-0.075457,0.136883,-0.038509,0.103257
1207,1,Q5VTA0,187,RGLVHLCCNKVQNYSMPTSSFRNLL,N,0.053669,-0.124710,-0.361735,-0.071841,0.163347,...,0.001311,-0.028484,-0.032336,0.126374,0.163849,0.072360,0.196219,0.107155,0.237708,0.201554


In [3]:
glyco_atlas = pd.read_csv("/home/d/repos/protein_properties/data/LMNglyPred/Train_Data_NGlycositeAtlas.csv")
glyco_atlas

,PID,Position,Sequence,Middle_Amino_Acid_ASN(N),label
0,Q13425,167,RALRLGDAILSVNGTDLRQATHDQA,N,1
1,Q9NZR2,1921,TSFAVGIDFHAENDTIYWTDMGFNK,N,1
2,O43663,489,LAPNTPGKARKLNTTTMSNATANSS,N,1
3,Q495M3,181,LADNLKQVVEAVNSTTNNCYSNETV,N,1
4,Q9BXP8,1408,CPSLLLDHADVVNCTSIGPGLMKCA,N,1
...,...,...,...,...,...
24260,O60061,307,ISMAIDLLVIRLNYTFGIWERPSAK,N,0
24261,Q96RL7,181,RDKPLSFGISLQNLSMQTTDQYWVP,N,0
24262,Q9Y3S1,1921,SFHSQSSYISSDNDSELEDADIKKE,N,0
24263,O70511,1616,ITEATPIKGVAPNSTLSSRTSPVTT,N,0


In [6]:
len(set(glyco_atlas["PID"]).intersection(set(df_train["PID"])))

1040

In [5]:
len(df_train["PID"])

1209

In [10]:
ngly_df = pd.read_csv("/home/d/repos/protein_properties/data/LMNglyPred/Glycobiology_NGlyDE_Training_Positive_1821_Negative_901.csv", usecols=[1])
set(ngly_df)
# wtf is this shit man ? 

{'A6NH11'}

In [11]:
taher_df = pd.read_csv("/home/d/repos/protein_properties/data/N_taherzadeh/Datasets.csv")

In [14]:
set(taher_df["Protein name"]).intersection(set(df_train["PID"]))

set()

In [2]:
from Bio import ExPASy
from Bio import SwissProt

def fetch_protein_sequence(uniprot_id):
    handle = ExPASy.get_sprot_raw(uniprot_id)
    record = SwissProt.read(handle)
    return record.sequence

fetch_protein_sequence("Q92954")
# position of glyco site is one based 

'MAWKTLPIYLLLLLSVFVIQQVSSQDLSSCAGRCGEGYSRDATCNCDYNCQHYMECCPDFKRVCTAELSCKGRCFESFERGRECDCDAQCKKYDKCCPDYESFCAEVHNPTSPPSSKKAPPPSGASQTIKSTTKRSPKPPNKKKTKKVIESEEITEEHSVSENQESSSSSSSSSSSSTIRKIKSSKNSAANRELQKKLKVKDNKKNRTKKKPTPKPPVVDEAGSGLDNGDFKVTTPDTSTTQHNKVSTSPKITTAKPINPRPSLPPNSDTSKETSLTVNKETTVETKETTTTNKQTSTDGKEKTTSAKETQSIEKTSAKDLAPTSKVLAKPTPKAETTTKGPALTTPKEPTPTTPKEPASTTPKEPTPTTIKSAPTTPKEPAPTTTKSAPTTPKEPAPTTTKEPAPTTPKEPAPTTTKEPAPTTTKSAPTTPKEPAPTTPKKPAPTTPKEPAPTTPKEPTPTTPKEPAPTTKEPAPTTPKEPAPTAPKKPAPTTPKEPAPTTPKEPAPTTTKEPSPTTPKEPAPTTTKSAPTTTKEPAPTTTKSAPTTPKEPSPTTTKEPAPTTPKEPAPTTPKKPAPTTPKEPAPTTPKEPAPTTTKKPAPTTPKEPAPTTPKETAPTTPKKLTPTTPEKLAPTTPEKPAPTTPEELAPTTPEEPTPTTPEEPAPTTPKAAAPNTPKEPAPTTPKEPAPTTPKEPAPTTPKETAPTTPKGTAPTTLKEPAPTTPKKPAPKELAPTTTKEPTSTTSDKPAPTTPKGTAPTTPKEPAPTTPKEPAPTTPKGTAPTTLKEPAPTTPKKPAPKELAPTTTKGPTSTTSDKPAPTTPKETAPTTPKEPAPTTPKKPAPTTPETPPPTTSEVSTPTTTKEPTTIHKSPDESTPELSAEPTPKALENSPKEPGVPTTKTPAATKPEMTTTAKDKTTERDLRTTPETTTAAPKMTKETATTTEKTTESKITATTTQVTSTTTQDTTPFKITTLKTTTLAPKVTTTKKTITTTEIMN

### O-Glyco sites 

In [3]:
headers = []
with open("/home/d/repos/protein_properties/data/O_captor/Ptrain.fasta", "r") as f:
    for line in f.readlines():
        if line.startswith(">"):
            headers.append(line.strip("\t").strip("\s").strip("\s")[4:])
            

In [4]:
import re
import numpy as np
rgx = re.compile('[%s]' % "\s\t|")
header_split = rgx.sub("", headers[0]).split("#")
seq = fetch_protein_sequence(header_split[0])


In [16]:
glyco_array_postions = np.array([int(i) for i in header_split[1:]]) - 1 
labels = np.zeros(len(seq))
labels[glyco_array_postions] = 1

True

In [19]:
# map dictonary to labels
label_dict = {0: "X", 1: "O"}
labels = [label_dict[i] for i in labels]
labels

['X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'O',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'O',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X',
 'X'